In [4]:
import torch
import torch.utils.data
from torchvision.utils import save_image
from torch import optim
import numpy as np
from models.GreyUTKFace import VAE
from models.GreyUTKFace import AgeClassifier
import os
from processed_data import GreyUTKFace
from utils import device
import matplotlib.pyplot as plt
plt.switch_backend('agg')

In [2]:
test_dataset = GreyUTKFace.Dataset(train=False, sample=False)

In [3]:
test_dataset.min_age

1

In [4]:
test_dataset.max_age

115

In [5]:
model = VAE.Model()
model.to(device)
model.load()

## Get Latent Point Average For Each Age

In [6]:
latent_ages = []

latent_ages.append(np.zeros(model.latent_space))

for age in range(test_dataset.min_age, test_dataset.max_age + 1):
    items = test_dataset.get_faces_of_age(age)
    faces = list(map(lambda item: item[0].numpy(), items))
    if len(faces) == 0:
        latent_ages.append(np.zeros(model.latent_space))
        continue
    faces = np.array(faces)
    face_imgs = torch.from_numpy(faces).to(device)
    means, _ = model.encode(face_imgs)
    latent_ages.append(np.average(means.detach().cpu().numpy(), axis=0))

## Generate Many Steps between Ages

In [ ]:
steps_per_age = 10
smooth_latent_ages = []
for i in range(len(latent_ages) - 1):
    age = latent_ages[i]
    next_age = latent_ages[i+1]
    
    smooth_latent_ages.append(age)
    
    if steps_per_age <= 1:
        continue
        
    step_vector = next_age - age / steps_per_age
    curr = age
    for i in range(steps_per_age - 1):
        curr += step_vector
        smooth_latent_ages.append(curr)   

In [7]:
tensor_latent_ages = torch.from_numpy(np.array(smooth_latent_ages)).to(device).float()
tensor_latent_ages

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.6775, -0.0817, -0.7176,  ..., -0.1975, -0.6193, -0.0118],
        [ 0.7834, -0.1547, -0.6889,  ..., -0.0632, -0.4801,  0.0488],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3699, -0.1727, -0.7827,  ..., -0.3762, -0.1263, -1.0620]],
       device='cuda:0')

In [8]:
face_ages = model.decode(tensor_latent_ages).cpu()

In [12]:
torch.save(model.state_dict(), model.dir + 'weights/model.pt')

In [10]:
try:
    os.makedirs(model.dir + 'results/gif/')
except:
    pass
for i in range(len(tensor_latent_ages)):
    plt.figure()
    plt.imshow(face_ages[i].view(model.input_w, model.input_h).detach().numpy())
    plt.title('Age: {}'.format(round(i / steps_per_age, 2)))
    plt.savefig(model.dir + 'results/gif/{}.png'.format(i))
    plt.close()

## Make a GIF

In [11]:
import glob
import os

gif_path = model.dir + 'results/age.gif'
img_list_file_path = model.dir + 'results/gif/image_list.txt'
file_list = glob.glob(model.dir + 'results/gif/*.png') # Get all the pngs in the current directory
list.sort(file_list, key=lambda x: int(x.split('.png')[0].split('/')[-1])) # Sort the images by #, this may need to be tweaked for your use case

with open(img_list_file_path, 'w') as file:
    for item in file_list:
        file.write("%s\n" % item)
command = 'convert @{} {}'.format(img_list_file_path, gif_path)
print(command)
os.system(command)

convert @models/GreyUTKFace/VAE/results/gif/image_list.txt models/GreyUTKFace/VAE/results/age.gif


0

# Age Classifier Quantitative Evaluation

In [5]:
age_classifier = AgeClassifier.Model()
age_classifier.to(device)
age_classifier.load()

Model(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
    (5): ReLU()
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
    (7): ReLU()
  )
  (fc1): Linear(in_features=12544, out_features=1, bias=True)
)
